In [1]:
import tensorflow as tf
import keras
import random
from pathlib import Path 
from time import strftime 

2026-02-05 22:16:17.640763: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-05 22:16:17.649450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-02-05 22:16:17.660265: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-02-05 22:16:17.663769: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-02-05 22:16:17.672008: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Load the dataset
(X_train_all, y_train_all), (X_test, y_test) = keras.datasets.cifar10.load_data()

X_valid, y_valid = X_train_all[-5000:], y_train_all[-5000:]
X_train, y_train = X_train_all[:-5000], y_train_all[:-5000]


In [ ]:
X_train.shape

In [ ]:
print(X_train[0, 0:32, 0, 0]) # integers between zero and 255 

In [3]:
X_train = (X_train / 255.)
X_valid = (X_valid / 255.) 
X_test = (X_test / 255.)

In [4]:
param_dict = {
    'learning_rate' : [0.0001, 0.0005, 0.001, 0.005, 0.01], 
    'batch_size' : [16, 32, 64, 128], 
    'epochs' : [50, 100, 150, 200], 
    'num_layers' : [2, 4, 6, 8, 10],
    'neurons' : [200, 400, 600],
    'optimizer' : ['adam', 'sgd'],
    'lr_sched' : ['exp', 'poly']
}

In [5]:
## this is the function for the random model
def random_model(neurons=128, num_layers=2):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=[32, 32, 3]))
    model.add(tf.keras.layers.Flatten())
    for _ in range(num_layers):
        model.add(tf.keras.layers.Dense(neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    return model

In [ ]:
def other_model(neurons):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=[32, 32, 3]))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    return model

In [ ]:
results2 = []
num_trials = 10
for trial in range(num_trials):

    lr_og = 0.0005
    
    end_learning_rate = 0.01
    decay_steps = 10000
    lr_schedule = keras.optimizers.schedules.PolynomialDecay(
        lr_og,
        decay_steps,
        end_learning_rate,
        power=0.5)
    
    model = other_model(neurons=600) # creating the  model

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

    
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    
    run_logdir = Path("my_logs/manual2") / f"trial_{trial}"
    tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=100)
    
    history = model.fit(
        X_train, y_train, 
        epochs=200,
        validation_data=(X_valid, y_valid), 
        callbacks=[tensorboard_cb, early_stopping_cb],
        batch_size=128
    )
    
    results2.append({
        'final_val_acc' : max(history.history['val_accuracy']),
        'final_train_acc' : max(history.history['accuracy']),
        'run_id' : trial
    })

In [ ]:
results2

In [6]:
def get_hyperparams(param_dict):
    list = []
    for key, values in param_dict.items():
        dict = {key : random.choice(values) for key, values in param_dict.items()}
        return dict

In [ ]:
def get_run_logdir(root_logdir="my_logs"):
    return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")

run_logdir = get_run_logdir()

In [7]:
results = []
num_trials = 10
for trial in range(num_trials):
    hps = get_hyperparams(param_dict)
    print(f"Hyperparams for trial {trial}: {hps}")

    lr_og = hps["learning_rate"] # original learning rate for lr schedulers
    lr_sched = hps["lr_sched"] # picking a random lr scheduler
    
    if lr_sched == "exp": 
        lr_schedule = keras.optimizers.schedules.ExponentialDecay(
            lr_og,
            decay_steps=100000,
            decay_rate=0.96,
            staircase=True)
    else:
        end_learning_rate = 0.01
        decay_steps = 10000
        lr_schedule = keras.optimizers.schedules.PolynomialDecay(
            lr_og,
            decay_steps,
            end_learning_rate,
            power=0.5)
    
    model = random_model(neurons=hps['neurons'], num_layers=hps["num_layers"]) # creating the random model

    opt = hps["optimizer"] # picking the random optimizer
    if opt == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)
    
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    
    run_logdir = Path("my_logs/manual3") / f"trial_{trial}"
    tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=30)
    
    history = model.fit(
        X_train, y_train, 
        epochs=hps['epochs'],
        validation_data=(X_valid, y_valid), 
        callbacks=[tensorboard_cb, early_stopping_cb],
        batch_size=hps['batch_size'],
    )
    
    results.append({
        'hyperparams' : hps,
        'final_val_acc' : max(history.history['val_accuracy']),
        'final_train_acc' : max(history.history['accuracy']),
        'run_id' : trial
    })

Hyperparams for trial 0: {'learning_rate': 0.01, 'batch_size': 64, 'epochs': 150, 'num_layers': 4, 'neurons': 400, 'optimizer': 'sgd', 'lr_sched': 'poly'}


I0000 00:00:1770347806.029380  642763 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1770347806.076564  642763 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1770347806.080371  642763 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1770347806.087598  642763 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Epoch 1/150


I0000 00:00:1770347807.906991  643367 service.cc:146] XLA service 0x7381ac004800 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1770347807.907012  643367 service.cc:154]   StreamExecutor device (0): NVIDIA RTX A400, Compute Capability 8.6
2026-02-05 22:16:47.918279: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2026-02-05 22:16:47.951941: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 91700
2026-02-05 22:16:48.538766: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_63', 68 bytes spill stores, 68 bytes spill loads

2026-02-05 22:16:48.844681: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_

 92/704 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1789 - loss: 2.1887

...

352/352 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.1320 - loss: 2.7859 - val_accuracy: 0.2200 - val_loss: 2.0204
Epoch 2/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2590 - loss: 1.9647 - val_accuracy: 0.3028 - val_loss: 1.8552
Epoch 3/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3084 - loss: 1.8579 - val_accuracy: 0.3232 - val_loss: 1.8335
Epoch 4/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3320 - loss: 1.8099 - val_accuracy: 0.3558 - val_loss: 1.7739
Epoch 5/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3602 - loss: 1.7592 - val_accuracy: 0.3280 - val_loss: 1.8213
Epoch 6/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3609 - loss: 1.7497 - val_accuracy: 0.3858 - val_loss: 1.7376
Epoch 7/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3779 - loss: 1.7076 - val_accuracy: 0.3910 - val_loss: 1.7117
Epoch 8/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3865 - loss: 1.6849 - val_accuracy: 0.3628 - val

2026-02-05 22:31:06.171204: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_222', 56 bytes spill stores, 76 bytes spill loads



352/352 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.2156 - loss: 2.1634 - val_accuracy: 0.3228 - val_loss: 1.9110
Epoch 2/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3423 - loss: 1.8670 - val_accuracy: 0.3696 - val_loss: 1.8174
Epoch 3/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3758 - loss: 1.7771 - val_accuracy: 0.3902 - val_loss: 1.7464
Epoch 4/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3979 - loss: 1.7122 - val_accuracy: 0.4042 - val_loss: 1.6971
Epoch 5/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4179 - loss: 1.6568 - val_accuracy: 0.4214 - val_loss: 1.6627
Epoch 6/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4318 - loss: 1.6255 - val_accuracy: 0.4250 - val_loss: 1.6224
Epoch 7/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4453 - loss: 1.5782 - val_accuracy: 0.4466 - val_loss: 1.5823
Epoch 8/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4511 - loss: 1.5551 - val_accuracy: 0.4494 - val

2026-02-05 22:31:42.937863: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_71', 248 bytes spill stores, 248 bytes spill loads

2026-02-05 22:31:43.245608: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_71', 64 bytes spill stores, 64 bytes spill loads

2026-02-05 22:31:43.489259: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_362', 112 bytes spill stores, 112 bytes spill loads

2026-02-05 22:31:43.600433: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_141', 312 bytes spill stores, 312 bytes spill loads

2026-02-05 22:31:43.652610: I external/local_xla/xla/stream_

704/704 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.2617 - loss: 2.0103 - val_accuracy: 0.3548 - val_loss: 1.8003
Epoch 2/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3887 - loss: 1.6897 - val_accuracy: 0.4076 - val_loss: 1.6653
Epoch 3/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4257 - loss: 1.5859 - val_accuracy: 0.4440 - val_loss: 1.5530
Epoch 4/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4569 - loss: 1.5236 - val_accuracy: 0.4392 - val_loss: 1.5476
Epoch 5/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4693 - loss: 1.4737 - val_accuracy: 0.4570 - val_loss: 1.5129
Epoch 6/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4880 - loss: 1.4222 - val_accuracy: 0.4864 - val_loss: 1.4336
Epoch 7/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5088 - loss: 1.3707 - val_accuracy: 0.4602 - val_loss: 1.4835
Epoch 8/150
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5213 - loss: 1.3352 - val_accuracy: 0.4966

In [10]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs/manual2

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [8]:
results

[{'hyperparams': {'learning_rate': 0.01,
   'batch_size': 64,
   'epochs': 150,
   'num_layers': 4,
   'neurons': 400,
   'optimizer': 'sgd',
   'lr_sched': 'poly'},
  'final_val_acc': 0.5307999849319458,
  'final_train_acc': 0.8512444496154785,
  'run_id': 0},
 {'hyperparams': {'learning_rate': 0.005,
   'batch_size': 64,
   'epochs': 100,
   'num_layers': 10,
   'neurons': 200,
   'optimizer': 'sgd',
   'lr_sched': 'exp'},
  'final_val_acc': 0.5325999855995178,
  'final_train_acc': 0.731844425201416,
  'run_id': 1},
 {'hyperparams': {'learning_rate': 0.0001,
   'batch_size': 16,
   'epochs': 50,
   'num_layers': 2,
   'neurons': 600,
   'optimizer': 'adam',
   'lr_sched': 'exp'},
  'final_val_acc': 0.5540000200271606,
  'final_train_acc': 0.8779777884483337,
  'run_id': 2},
 {'hyperparams': {'learning_rate': 0.0005,
   'batch_size': 64,
   'epochs': 200,
   'num_layers': 6,
   'neurons': 400,
   'optimizer': 'sgd',
   'lr_sched': 'exp'},
  'final_val_acc': 0.5436000227928162,
  'fina